# RQ3: 

In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial import distance

pd.options.display.float_format = '{:,.4f}'.format

In [23]:
base_dir = "../trained_models/BagOfTricks/1000val/full/"
def load_data(train, test, fname, valtest=None) :
    if valtest == None :
        fpath = base_dir + "{}/eval/best/{}/{}.txt".format(train, test, fname)
    else :
        fpath = base_dir + "{}/val/{}/best/{}/{}.txt".format(train, valtest, test, fname)

    return np.loadtxt(fpath).astype(np.int)

train = "autoattack"
test = "pgd"

y_original = load_data(train, test, "Y_original", 1000)
y_original_pred = load_data(train, test, "Y_original_pred", 1000)
y_adv = load_data(train, test, "Y_adv", 1000)
y_adv_pred = load_data(train, test, "Y_adv_pred", 1000)

In [24]:
def get_robust_data(train, test, valtest=None):
    y_adv = load_data(train, test, "Y_adv", valtest)
    y_adv_pred = load_data(train, test, "Y_adv_pred", valtest)
    return y_adv, y_adv_pred

train = "pgd"
test = "autoattack"
y_adv, y_adv_pred = get_robust_data(train, test, None)

print("Y_adv({},{}): {}".format(train, test, y_adv))
print("Y_adv_pred({},{}): {}".format(train, test, y_adv_pred))

Y_adv(pgd,autoattack): [3 8 8 ... 5 1 7]
Y_adv_pred(pgd,autoattack): [3 8 8 ... 5 1 7]


In [49]:
attacks = ["autoattack", "autopgd", "bim", "cw", "fgsm", "pgd", "deepfool", "newtonfool", "squareattack"]

In [50]:
train = attacks.copy()

test = attacks.copy()
# test.append("all")

In [51]:
def get_accuracies(train, test, valtest) :
    predictions = {}
    labels = {}

    for a1 in train :
        predictions[a1] = {}
        labels[a1] = {}
        
    for a1 in train :
        for a2 in test :
            y, y_pred = get_robust_data(a1, a2, valtest)
            predictions[a1][a2] = y_pred
            labels[a1][a2] = y

    accs = {}
    for tr in train :
        accs[tr] = {}
        for ts in test :
            accs[tr][ts] = (labels[tr][ts] == predictions[tr][ts]).sum().item() / len(labels[tr][ts])


    accs = pd.DataFrame(data=accs)
    accs_transpose = pd.DataFrame(data=accs).T
    return accs_transpose, accs 

valtest = None
accs_transpose, accs = get_accuracies(train, test, valtest)

## Centroid For the Cluster 0

In [52]:
accsT, accs = get_accuracies(train, test, None)
accsT["all"] = accs.mean()
accsT

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,all
autoattack,0.7144,0.7126,0.7104,0.7204,0.6923,0.7113,0.7068,0.7193,0.7173,0.7116
autopgd,0.7182,0.7189,0.7124,0.7287,0.6979,0.7128,0.7112,0.7265,0.7222,0.7165
bim,0.7166,0.7190,0.7122,0.7256,0.6946,0.7138,0.7143,0.7239,0.7216,0.7157
cw,0.7035,0.7056,0.7034,0.7237,0.6800,0.7021,0.7079,0.7199,0.7154,0.7068
fgsm,0.7018,0.7033,0.7018,0.7096,0.6992,0.7022,0.7062,0.7099,0.7063,0.7045
pgd,0.7023,0.7034,0.6999,0.7123,0.6819,0.6993,0.6984,0.7114,0.7066,0.7017
deepfool,0.7164,0.7163,0.7104,0.7280,0.7019,0.7121,0.7212,0.7281,0.7179,0.7169
newtonfool,0.6988,0.7032,0.7016,0.7180,0.6847,0.6989,0.7023,0.7185,0.7173,0.7048
squareattack,0.6391,0.6398,0.6399,0.7056,0.6202,0.6351,0.6642,0.7004,0.7511,0.6662


In [53]:
sorted_acc = accsT.sort_values(by="all", ascending=False)
idx = list(sorted_acc.index)
sorted_acc = sorted_acc.reindex(idx, axis=0)
idx.append("all")
sorted_acc = sorted_acc.reindex(idx, axis=1)
sorted_acc

,deepfool,autopgd,bim,autoattack,cw,newtonfool,fgsm,pgd,squareattack,all
deepfool,0.7212,0.7163,0.7104,0.7164,0.7280,0.7281,0.7019,0.7121,0.7179,0.7169
autopgd,0.7112,0.7189,0.7124,0.7182,0.7287,0.7265,0.6979,0.7128,0.7222,0.7165
bim,0.7143,0.7190,0.7122,0.7166,0.7256,0.7239,0.6946,0.7138,0.7216,0.7157
autoattack,0.7068,0.7126,0.7104,0.7144,0.7204,0.7193,0.6923,0.7113,0.7173,0.7116
cw,0.7079,0.7056,0.7034,0.7035,0.7237,0.7199,0.6800,0.7021,0.7154,0.7068
newtonfool,0.7023,0.7032,0.7016,0.6988,0.7180,0.7185,0.6847,0.6989,0.7173,0.7048
fgsm,0.7062,0.7033,0.7018,0.7018,0.7096,0.7099,0.6992,0.7022,0.7063,0.7045
pgd,0.6984,0.7034,0.6999,0.7023,0.7123,0.7114,0.6819,0.6993,0.7066,0.7017
squareattack,0.6642,0.6398,0.6399,0.6391,0.7056,0.7004,0.6202,0.6351,0.7511,0.6662


## Best Accuracy Using 1 Model

In [54]:
attacks = ["autoattack", "autopgd", "bim", "cw", "fgsm", "pgd", "deepfool", "newtonfool", "squareattack", "pixelattack", "spatialtransformation"]

In [56]:
train = attacks.copy()
test = attacks.copy()
accsT, accs = get_accuracies(train, test, None)
accsT["all"] = accs.mean()
accsT

,autoattack,autopgd,bim,cw,fgsm,pgd,deepfool,newtonfool,squareattack,pixelattack,spatialtransformation,all
autoattack,0.7144,0.7126,0.7104,0.7204,0.6923,0.7113,0.7068,0.7193,0.7173,0.1036,0.3316,0.6218
autopgd,0.7182,0.7189,0.7124,0.7287,0.6979,0.7128,0.7112,0.7265,0.7222,0.1428,0.3648,0.6324
bim,0.7166,0.7190,0.7122,0.7256,0.6946,0.7138,0.7143,0.7239,0.7216,0.0931,0.2924,0.6206
cw,0.7035,0.7056,0.7034,0.7237,0.6800,0.7021,0.7079,0.7199,0.7154,0.1083,0.3682,0.6216
fgsm,0.7018,0.7033,0.7018,0.7096,0.6992,0.7022,0.7062,0.7099,0.7063,0.1054,0.2824,0.6116
pgd,0.7023,0.7034,0.6999,0.7123,0.6819,0.6993,0.6984,0.7114,0.7066,0.1223,0.3381,0.6160
deepfool,0.7164,0.7163,0.7104,0.7280,0.7019,0.7121,0.7212,0.7281,0.7179,0.0913,0.3472,0.6264
newtonfool,0.6988,0.7032,0.7016,0.7180,0.6847,0.6989,0.7023,0.7185,0.7173,0.1532,0.3257,0.6202
squareattack,0.6391,0.6398,0.6399,0.7056,0.6202,0.6351,0.6642,0.7004,0.7511,0.1064,0.3089,0.5828
pixelattack,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.6440,0.1000,0.1495


In [57]:
sorted_acc = accsT.sort_values(by="all", ascending=False)
idx = list(sorted_acc.index)
sorted_acc = sorted_acc.reindex(idx, axis=0)
idx.append("all")
sorted_acc = sorted_acc.reindex(idx, axis=1)
sorted_acc

,autopgd,deepfool,autoattack,cw,bim,newtonfool,pgd,fgsm,squareattack,spatialtransformation,pixelattack,all
autopgd,0.7189,0.7112,0.7182,0.7287,0.7124,0.7265,0.7128,0.6979,0.7222,0.3648,0.1428,0.6324
deepfool,0.7163,0.7212,0.7164,0.7280,0.7104,0.7281,0.7121,0.7019,0.7179,0.3472,0.0913,0.6264
autoattack,0.7126,0.7068,0.7144,0.7204,0.7104,0.7193,0.7113,0.6923,0.7173,0.3316,0.1036,0.6218
cw,0.7056,0.7079,0.7035,0.7237,0.7034,0.7199,0.7021,0.6800,0.7154,0.3682,0.1083,0.6216
bim,0.7190,0.7143,0.7166,0.7256,0.7122,0.7239,0.7138,0.6946,0.7216,0.2924,0.0931,0.6206
newtonfool,0.7032,0.7023,0.6988,0.7180,0.7016,0.7185,0.6989,0.6847,0.7173,0.3257,0.1532,0.6202
pgd,0.7034,0.6984,0.7023,0.7123,0.6999,0.7114,0.6993,0.6819,0.7066,0.3381,0.1223,0.6160
fgsm,0.7033,0.7062,0.7018,0.7096,0.7018,0.7099,0.7022,0.6992,0.7063,0.2824,0.1054,0.6116
squareattack,0.6398,0.6642,0.6391,0.7056,0.6399,0.7004,0.6351,0.6202,0.7511,0.3089,0.1064,0.5828
spatialtransformation,0.4444,0.4421,0.4405,0.4802,0.4362,0.4801,0.4405,0.4460,0.4994,0.7383,0.0983,0.4496


## Best Performing Model

In [65]:
# 3 classes
clusters = {'deepfool': ['autoattack', 'autopgd', 'bim', 'cw', 'fgsm', 'pgd', 'deepfool', 'newtonfool', 'squareattack'], 'pixelattack': ['pixelattack'], 'spatialtransformation': ['spatialtransformation']}

In [66]:
accsT, accs = get_accuracies(train, test, None)

In [67]:
accsT["all"] = accs.mean()

In [68]:
accs = accsT.T

In [70]:
def measure_best_performing_model(clusters):
    res = {}
    upper_bound = 0
    n = 0
    for centroid, members in clusters.items() :
        for m in members :
            res[m] = accs[centroid][m]
    return res
    
bpm = measure_best_performing_model(clusters)
bpm["all"] = np.array(list(bpm.values())).mean()

In [71]:
bpm

{'autoattack': 0.7164,
 'autopgd': 0.7163,
 'bim': 0.7104,
 'cw': 0.728,
 'fgsm': 0.7019,
 'pgd': 0.7121,
 'deepfool': 0.7212,
 'newtonfool': 0.7281,
 'squareattack': 0.7179,
 'pixelattack': 0.644,
 'spatialtransformation': 0.7383,
 'all': 0.7122363636363637}